#  Track Analysis

In order to properly analyze our GPX tracks, we need to put all of them into the same _frame-of-reference_.   This is accomplished by the following.

* Given a starting point and ending point, we can determine the best route through the  

## Step 0: Globals

In [1]:
database_path = 'bike_data.db'

start_coord = (39.5989743, -104.8609468)
end_coord   = (39.75428108249532, -105.00085402872664)

dist_thresh_m = 20
step_dist_m = 25

## Step 1: Import Required Libraries

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from ipyleaflet import Map, Marker, Polygon, Polyline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math, cmath
from geographiclib.geodesic import Geodesic

Setup the database.

In [3]:
# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT dataset_id FROM point_list', conn)

## Step 2: Import points 

In [4]:
#  For each dataset, load the points w.r.t. each dataset-id, sorted by time.
points_by_dataset = {}
for dataset_id in dataset_ids['dataset_id']:
        
    #  Create a full track of the segment
    sql_query = 'SELECT * FROM point_list WHERE dataset_id = {} ORDER BY timestamp'.format( dataset_id )
    points_by_dataset[dataset_id] = { 'points': pd.read_sql_query( sql_query, conn ) }

## Step 3: Compute Initial Starting Point

In [8]:
def mean_angle(angles):
    return math.degrees(cmath.phase(sum(cmath.rect(1, math.radians(d)) for d in angles)/len(angles)))

waypoint_list = []
current_point = start_coord
matching_points = []

current_idx = {}
for dataset_id in dataset_ids['dataset_id']:
    current_idx[dataset_id] = 0


#  Start a loop, running until all points are complete
while True:
    
    #  Look for the nearest first points within the threshold
    temp_angles = []
    for dataset_id in dataset_ids['dataset_id']:
        current_point = ( points_by_dataset[dataset_id]['points']['latitude'][current_idx[dataset_idx]],
                          points_by_dataset[dataset_id]['points']['longitude'][current_idx[dataset_idx]] )
    
        geod = Geodesic.WGS84.Inverse( start_coord[0], start_coord[1], current_point[0], current_point[1] )
        print(geod)
    
        if geod['s12'] < dist_thresh_m:
        
            #  Compute angle to new point
            temp_angles.append( geod['azi1'])
            print('Using Point from Dataset: {}. Dist: {}, Azimuth: {}'.format(dataset_id, geod['s12'], geod['azi1']))
            current_route[dataset_id] = 0
            matching_points.append( (dataset_id, 0) )
        else:
            current_route[dataset_id] = -1
    avg_angle = mean_angle( temp_angles )
    print('Azimuth: {}'.format(avg_angle))

    #  Compute new point
    geod = Geodesic.WGS84.Direct( current_point[0],
                                  current_point[1],
                                  avg_angle,
                                  step_dist_m )
    current_point = ( geod['lat2'], geod['lon2'] )
    
    #  Update Current Indices
    for dataset_id in dataset_ids:
        pass
    

NameError: name 'dataset_idx' is not defined